# Ollama 自定义导入模型

## 从 GGUF 导入


GGUF (GPT-Generated Unified Format) 是一种文件格式，用于保存经过微调的语言模型。  
这种格式旨在帮助用户方便地在不同的平台和环境之间共享和导入模型。  
它支持多种量化格式，可以有效减少模型文件的大小。  


它的前身是 GGML(GPT-Generated Model Language)，是专门为了机器学习而设计的 Tensor 库，目的是为了有一个单文件的格式，并且易在不同架构的 CPU 以及 GPU 上可以推理，但后续由于开发遇到了灵活性不足、相容性及难以维护的问题。


### 下载 .gguf 文件


在 huggingface 上找到具体的 gguf 文件，点击下载  

例如： https://huggingface.co/RichardErkhov/Qwen_-_Qwen2-0.5B-gguf

### 新建创建 Modelfile 文件


```
FROM ./Qwen2-0.5B.Q3_K_M.gguf

```

### 在 Ollama 中创建模型


```bash
ollama create Qwen2-0.5B -f Modelfile
```

### 终端内运行模型

```bash
ollama run Qwen2-0.5B
```

## 从 Pytorch 或 Safetensors 导入



Safetensors 是一种用于存储深度学习模型权重的文件格式，它旨在解决安全性、效率和易用性方面的问题。目前这部分功能还有待社区成员开发，目前文档资源有限。



### 下载模型

安装依赖 `pip install huggingface_hub`  
https://huggingface.co/docs/huggingface_hub/main/en/guides/download

In [1]:
# 下载模型
import os
from huggingface_hub import snapshot_download

# 获取地址: https://huggingface.co/settings/tokens
HUGGING_FACE_API_KEY = os.environ['HUGGING_FACE_API_KEY']

model_id = "unsloth/Llama-3.2-1B-bnb-4bit"
snapshot_download(
  repo_id=model_id, 
  local_dir="Llama-3.2-1B-bnb-4bit",
  local_dir_use_symlinks=False,
  revision="main",
  use_auth_token=HUGGING_FACE_API_KEY)


/Users/tiankonguse-m3/miniconda3/envs/llm-study/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tiankonguse-m3/miniconda3/envs/llm-study/lib/python3.12/site-packages/huggingface_hub/file_download.py:834: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00,  9.03it/s]


'/Users/tiankonguse-m3/project/github/llm-study/ollama/Llama-3.2-1B-bnb-4bit'

### 创建 Modelfile 文件

```
FROM ./llama-3-8b-bnb-4bit
```

### 创建模型

```bash
# Create the model in Ollama
ollama create llama-3-8b-bnb-4bit -f Modelfile
```

### 运行模型

```bash
# Run the model
ollama run llama-3-8b-bnb-4bit
```


## 模型直接导入


正常来说，我们在 HuggingFace 接触到的模型文件非常之多，庆幸的是，hf 提供了非常便捷的 API 来下载和处理这些模型，像上面那样直接下载受限于网络环境，速度非常慢，这一小段我们来使用脚本以及 hf 来完成。

llama.cpp 是 GGUF 的开源项目，提供 CLI 和 Server 功能。

对于不能通过 Ollama 直接转换的架构，我们可以使用 llama.cpp 进行量化，并将其转换为 GGUF 格式，再按照第一种方式进行导入。 我们整个转换的过程分为以下几步：

- 从 huggingface 上下载 model；
- 使用 llama.cpp 来进行转化；
- 使用 llama.cpp 来进行模型量化；
- 运行并上传模型。


### 从 HuggingFace 下载 Model


最直觉的下载方式是通过git clone或者链接来下载，但是因为llm每部分都按GB计算，避免出现 OOM Error(Out of memory)，我们可以使用 Python 写一个简单的 download.py。





In [2]:
# 下载模型
import os
from huggingface_hub import snapshot_download

# 获取地址: https://huggingface.co/settings/tokens
HUGGING_FACE_API_KEY = os.environ['HUGGING_FACE_API_KEY']

model_id = "Qwen/Qwen1.5-0.5B" # hugginFace's model name
snapshot_download(
    repo_id=model_id, 
    # local_dir="Qwen-0.5b",
    # local_dir_use_symlinks=False,
    revision="main",
    use_auth_token=HUGGING_FACE_API_KEY)

Fetching 10 files: 100%|██████████| 10/10 [05:11<00:00, 31.16s/it]


'/Users/tiankonguse-m3/.cache/huggingface/hub/models--Qwen--Qwen1.5-0.5B/snapshots/8f445e3628f3500ee69f24e1303c9f10f5342a39'

### 使用 llama.cpp 进行转换

lama.cpp 是 GGML 主要作者基于最早的 llama 的 c/c++ 版本开发的，目的就是希望用 CPU 来推理各种 LLM，在社群的不断努力下现在已经支持大多数主流模型，甚至包括多模态模型。

首先我们克隆 llama.cpp 库到本地，与下载的模型放在同一目录下：

```bash
git clone https://github.com/ggerganov/llama.cpp.git
```

由于使用 llama.cpp 转换模型的流程基于 python 开发，需要安装相关的库，推荐使用 conda 或 venv 新建一个环境。

```bash
cd llama.cpp
pip install -r requirements.txt
python convert_hf_to_gguf.py -h
```

接下来，我们把刚刚从 HuggingFace 下载的模型转换为 GGUF 格式，具体使用以下脚本：

```bash
python convert_hf_to_gguf.py ./Qwen-0.5b --outfile Qwen_instruct_0.5b.gguf --outtype f16

```

可以看到 llama.cpp 目录下多了一个 Qwen_instruct_0.5b.gguf 文件，这个过程只需要几秒钟。

为了节省推理时的开销，我们将模型量化，接下来我们开始量化实操。



### 使用llama.cpp进行模型量化


模型量化是一种技术，将高精度的浮点数模型转换为低精度模型，模型量化的主要目的是减少模型的大小和计算成本，尽可能保持模型的准确性，其目标是使模型能够在资源有限的设备上运行，例如CPU或者移动设备。

同样的，我们先创建 Modelfile 文件，再使 用 ollama create 命令来从 gguf 文件中创建我们的模型，不过与第一步稍有不同的是，我们添加了量化逻辑，只需要在执行 ollama create 是添加一个参数即可。


首先把上一步拿到的 Qwen_instruct_0.5b.gguf 移动至第三部分的根目录下，再创建 Modelfile 文件编写以下内容.

```
FROM ./Qwen_instruct_0.5b.gguf
```

终端运行创建和量化脚本。

```
ollama create -q Q4_K_M mymodel3 -f ./Modelfile
```

### 上传模型




In [ ]:
from huggingface_hub import HfApi
import os

api = HfApi()
HF_ACCESS_TOKEN = "<YOUR_HF_WRITE_ACCESS_TOKEN>"
#TODO 这里需要设置你的model_id
#例如 model_id = "little1d/QWEN-0.5b"
model_id = "your_hf_name/QWEN-0.5b"


api.create_repo(
    model_id,
    exist_ok=True,
    repo_type="model", # 上傳格式為模型
    use_auth_token=HF_ACCESS_TOKEN,
)
# upload the model to the hub
# upload model name includes the Bailong-instruct-7B in same folder
for file in os.listdir():
    if file.endswith(".gguf"):
        model_name = file.lower()
        api.upload_file(
            repo_id=model_id,
            path_in_repo=model_name,
            path_or_fileobj=f"{os.getcwd()}/{file}",
            repo_type="model", # 上傳格式為模型
            use_auth_token=HF_ACCESS_TOKE)


## 自定义 Prompt

Ollama 支持自定 义Prompt，可以让模型生成更符合用户需求的文本。


### 根目录下创建一个 Modelfile 文件

```Modelfile
FROM llama3.1
# sets the temperature to 1 [higher is more creative, lower is more coherent]
PARAMETER temperature 1
# sets the context window size to 4096, this controls how many tokens the LLM can use as context to generate the next token
PARAMETER num_ctx 4096

# sets a custom system message to specify the behavior of the chat assistant
SYSTEM You are Mario from super mario bros, acting as an assistant.

```


### 创建模型

```
ollama create mymodel3 -f ./Modelfile
```